<a href="https://colab.research.google.com/github/Oneday37/skripsi_drug_recommendation_system/blob/main/Drug_Recommendation_Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
import pandas as pd # Data processing, CSV file I/O

from sklearn.feature_extraction.text import TfidfVectorizer # Mengubah teks yang awalnya bertipe data string menjadi bentuk vector numerik dengan metode TF-IDF
from sklearn.metrics.pairwise import cosine_similarity # Menghitung kemiripan antara 2 vektor menggunakan metode Cosine Similarity

## Import Dataset

In [ ]:
# Membaca file dataset dan menampilkan informasi ringkas terkait keseluruhan dataset
drugs_df = pd.read_csv('medicine_data.csv')
drugs_df.describe()

,Disease (Penyakit),Obat,Deskripsi,Kandungan,Dosis,Aturan Pakai,Efek Samping,Toko Online 1,Toko Online 2,Sumber,Images,Kategori
count,127,127,127,127,127,127,127,127,127,127,127,127
unique,46,88,94,81,103,66,83,86,85,68,88,6
top,Migrain,Amoxicillin,Amoxicillin merupakan sebuah obat antibiotik g...,Paracetamol,-,Dapat dikonsumsi sebelum atau sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/amoxicilli...,"Halodoc, Alodokter",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,Sakit Perut
freq,3,6,6,6,4,14,6,6,6,9,6,28


Pemanggilan fungsi **describe()** yang diikuti dengan variabel drugs_df merupakan funsi yang memiliki tujuan untuk menampilkan seluruh data yang tersimpan dalam dataset tersebut. Terdapat 4 baris kolom yang ditampilkan untuk setipa datanya, yakni:
1. Count = Baris yang akan menampilkan seluruh data yang tersimpan dalam dataset (Data null tidak akan dihitung dan ditampilkan)
2. Unique = Merupakan baris yang akan menampilkan jumlah dari data unik yang tersimpan pada dataset (jumlah data asli tanpa adanya duplicate)
3. Top = Baris yang akan menampilkan nilai dari dataset yang sering muncul
4. freq = Baris yang menampilkan frekuensi data yang diambil dari data **top**. Baris ini juga dapat digunakan dalam menentukan apakah pada dataset tersebut terdapat data yang diduplicate atau tidak

In [ ]:
# Menampilkan 10 data teratas
drugs_df.head(10)

,Disease (Penyakit),Obat,Deskripsi,Kandungan,Dosis,Aturan Pakai,Efek Samping,Toko Online 1,Toko Online 2,Sumber,Images,Kategori
0,Migrain,Bodrex Migra,Bodrex migra merupakan obat kaplet yang mengan...,"Paracetamol 350 mg, Propyphenazone 150 mg, dan...","Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\r\nhttps://www.halodoc.com/obat-dan-v...,\n\nAlodokter:\nhttps://www.alodokter.com/alos...,"Alodokter, Halodoc, Hellosehat, Klikdokter",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
1,Migrain,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,"Halodoc, Alodokter, Panadol, Hellosehat",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
2,Migrain,Farsifen Plus,Farsifen plus merupakan salah satu jenis varia...,"Paracetamol, Ibuprofen, dan Kafein","Dewasa dan anak-anak berusia > 12: 1 kaplet, 3...",Dapat dikonsumsi setalah makan untuk menghinda...,Obat tersebut memiliki beberapa efek samping y...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24Klik:\nhttps://www.k24klik.com/p/farsifen-p...,"Alodokter, Halodoc, dan K24Klik",https://res.cloudinary.com/dk0z4ums3/image/upl...,Sakit Kepala
3,Tension Headache,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,"Halodoc, Alodokter, Panadol, Hellosehat",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
4,Tension Headache,Paracetamol,Paracetamol atau dapat dikenal sebagai asetami...,Paracetamol,"1.000 mg, 4 kali/hari, selama 7 hari",Dapat dionsumsi sebelum atau setelah makan,Obat ini jarang menimbulkan efek samping kepad...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/paracetamo...,"Alodokter, Primayahospital, Alomdedika",https://pimpharma.com/wp-content/uploads/2023/...,Sakit Kepala
5,Tension Headache,Ibuprofen,Ibuprofen merupakan obat pereda nyeri non ster...,Ibuprofen,"Dewasa: 200 - 400 mg, 3x sehari\n\nAnak-anak: ...",Dapat dionsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,Alomedika,https://www.novapharin.co.id/data/plist_pic/96...,Sakit Kepala
6,Sinusitis,Amoxicillin,Amoxicillin merupakan sebuah obat antibiotik g...,Amoxicillin,Dewasa dan anak dengan BB ≥ 40 kg: Dosis amoxi...,Dapat dikonsumsi sebelum atau setelah makan. N...,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/amoxicilli...,"Alodokter, Halodoc, dan Primayahospital",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,Sakit Kepala
7,Sinusitis,Iliadin Nasal Spray,Iliadin merupakan obat yang mengandung oxymeta...,Oxymetazoline 0.5 mg dalam setiap 1 ml,"Dewasa dan anak-anak > 6 tahun: 2 kali sehari,...",Sebelum menyemprotkan obat ini ke dalam lubang...,Obat ini memiliki beberapa efek samping yang d...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,"Halodoc, Alodokter",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
8,Sinusitis,Breathy Nasal Drops,Breathy Tetes Hidung merupakan obat tetes yang...,Natrium Klorida,Dapat digunakan oleh anak-anak berusia > 1 bul...,Dapat diteteskan ke dalam masing-masing lubang...,Breathy Nasal Drops harus digunakan sesuai den...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\

## Data Cleaning

In [ ]:
# Menampilkan 5 data awal dan 5 data akhir
drugs_df

,Disease (Penyakit),Obat,Deskripsi,Kandungan,Dosis,Aturan Pakai,Efek Samping,Toko Online 1,Toko Online 2,Sumber,Images,Kategori
0,Migrain,Bodrex Migra,Bodrex migra merupakan obat kaplet yang mengan...,"Paracetamol 350 mg, Propyphenazone 150 mg, dan...","Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\r\nhttps://www.halodoc.com/obat-dan-v...,\n\nAlodokter:\nhttps://www.alodokter.com/alos...,"Alodokter, Halodoc, Hellosehat, Klikdokter",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
1,Migrain,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,"Halodoc, Alodokter, Panadol, Hellosehat",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
2,Migrain,Farsifen Plus,Farsifen plus merupakan salah satu jenis varia...,"Paracetamol, Ibuprofen, dan Kafein","Dewasa dan anak-anak berusia > 12: 1 kaplet, 3...",Dapat dikonsumsi setalah makan untuk menghinda...,Obat tersebut memiliki beberapa efek samping y...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24Klik:\nhttps://www.k24klik.com/p/farsifen-p...,"Alodokter, Halodoc, dan K24Klik",https://res.cloudinary.com/dk0z4ums3/image/upl...,Sakit Kepala
3,Tension Headache,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,"Halodoc, Alodokter, Panadol, Hellosehat",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Kepala
4,Tension Headache,Paracetamol,Paracetamol atau dapat dikenal sebagai asetami...,Paracetamol,"1.000 mg, 4 kali/hari, selama 7 hari",Dapat dionsumsi sebelum atau setelah makan,Obat ini jarang menimbulkan efek samping kepad...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/paracetamo...,"Alodokter, Primayahospital, Alomdedika",https://pimpharma.com/wp-content/uploads/2023/...,Sakit Kepala
...,...,...,...,...,...,...,...,...,...,...,...,...
122,Disentri,Metronidazole,Obat ini bekerja dengan membunuh parasit penye...,Metronidazole,"Dosis: 3 kali sehari, selama 5–10 hari.",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/metronidaz...,"Alodokter, Halodoc, Primayahospital, K24klik",https://s3-publishing-cmn-svc-prd.s3.ap-southe...,Sakit Perut
123,Hepatitis,-,-,-,-,-,-,-,-,-,-,Sakit Perut
124,Irritable Bowel Syndrome (IBS),Duspatalin,Duspatalin merupakan obat yang digunakan untuk...,Mebeverine Hydrochloride 135 mg,Dosis: 1 tablet dikonsumsi sebanyak 3x sehari,Dikonsumsi 20 menit sebelum makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/duspatalin...,"Alodokter, Halodoc, Hellosehat, Klikdokter, K2...",https://images.tokopedia.net/img/cache/700/OJW...,Sakit Perut
125,Irritable Bowel Syndrome (IBS),Dulcolax,Dulcolax adalah obat pencahar untuk mengatasi ...,Bisacodyl,Dewasa dan anak usia > 10 tahun: 1–2 tablet pe...,Dapat dikonsumsi sebelum tidur,Dulcolax memiliki beberapa efek samping yang d...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/dulcolax-t...,"Halodoc, Alodokter, Dulcolax, K24klik",https://d2qjkwm11akmwu.cloudfront.net/products...,Sakit Perut


In [ ]:
# Menampilkan informasi singkat berdasarkan kolom
drugs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Disease (Penyakit)  127 non-null    object
 1   Obat                127 non-null    object
 2   Deskripsi           127 non-null    object
 3   Kandungan           127 non-null    object
 4   Dosis               127 non-null    object
 5   Aturan Pakai        127 non-null    object
 6   Efek Samping        127 non-null    object
 7   Toko Online 1       127 non-null    object
 8   Toko Online 2       127 non-null    object
 9   Sumber              127 non-null    object
 10  Images              127 non-null    object
 11  Kategori            127 non-null    object
dtypes: object(12)
memory usage: 12.0+ KB


Menghilangkan White Space di bagian awal dan akhir setiap

In [ ]:
# Menghilangkan White Space di bagian awal dan akhir setiap dataset (termasuk \t atau \n)
drugs_df['Disease (Penyakit)'] = drugs_df['Disease (Penyakit)'].str.strip()
drugs_df['Obat'] = drugs_df['Obat'].str.strip()
drugs_df['Deskripsi'] = drugs_df['Deskripsi'].str.strip()
drugs_df['Kandungan'] = drugs_df['Kandungan'].str.strip()
drugs_df['Dosis'] = drugs_df['Dosis'].str.strip()
drugs_df['Aturan Pakai'] = drugs_df['Aturan Pakai'].str.strip()

In [ ]:
# Menampilkan informasi ringkas terkait keseluruhan dataset
drugs_df.describe()

,Disease (Penyakit),Obat,Deskripsi,Kandungan,Dosis,Aturan Pakai,Efek Samping,Toko Online 1,Toko Online 2,Sumber,Images,Kategori
count,127,127,127,127,127,127,127,127,127,127,127,127
unique,46,88,94,80,103,66,83,86,85,68,88,6
top,Migrain,Amoxicillin,Amoxicillin merupakan sebuah obat antibiotik g...,Paracetamol,-,Dapat dikonsumsi sebelum atau sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/amoxicilli...,"Halodoc, Alodokter",https://d1vbn70lmn1nqe.cloudfront.net/prod/wp-...,Sakit Perut
freq,3,6,6,6,4,14,6,6,6,9,6,28


In [ ]:
# Mengecek apakah ada data null di setiap column pada data frame
drugs_df.isnull().sum()

,0
Disease (Penyakit),0
Obat,0
Deskripsi,0
Kandungan,0
Dosis,0
Aturan Pakai,0
Efek Samping,0
Toko Online 1,0
Toko Online 2,0
Sumber,0


In [ ]:
# Memeriksa data penyakit yang ada pada dataset
print(f"Total data penyakit murni : {drugs_df['Disease (Penyakit)'].nunique()}")
print(f"Total data penyakit null : {drugs_df['Disease (Penyakit)'].isnull().sum()}")
print(f"Total data penyakit yang terduplikasi : {drugs_df['Disease (Penyakit)'].duplicated().sum()}")
print(f"Total data penyakit yang ada pada data frame : {len(drugs_df['Disease (Penyakit)'])}")

In [ ]:
# Menampilkan data unique (tanpa duplikasi) dari column Disease (Penyakit)
print(f"Total data penyakit murni : {drugs_df['Disease (Penyakit)'].nunique()}")
print(f"{drugs_df['Disease (Penyakit)'].unique()}")

In [ ]:
# Memeriksa data obat yang ada pada dataset
print(f"Total data obat murni : {drugs_df['Obat'].nunique()}")
print(f"Total data obat null : {drugs_df['Obat'].isnull().sum()}")
print(f"Total data obat yang terduplikasi : {drugs_df['Obat'].duplicated().sum()}")
print(f"Total data obat yang ada pada data frame : {len(drugs_df['Obat'])}")

In [ ]:
# Menampilkan data unique (tanpa duplikasi) dari column Obat
print(f"Total data obat murni : {drugs_df['Obat'].nunique()}")
print(f"{drugs_df['Obat'].unique()}")

In [ ]:
# Menampilkan daftar obat untuk penyakit bagian kepala
print(f"Total data obat untuk penyakit bagian kepala murni : {drugs_df[drugs_df['Kategori'] == 'Sakit Kepala']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Kepala']['Obat'].unique()}")

In [ ]:
# Menampilkan daftar obat untuk penyakit bagian telinga
print(f"Total data obat untuk penyakit bagian telinga murni : {drugs_df[drugs_df['Kategori'] == 'Sakit Telinga']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Telinga']['Obat'].unique()}")

In [ ]:
# Menampilkan daftar obat untuk penyakit bagian mata
print(f"Total data obat untuk penyakit bagian mata murni : {drugs_df[drugs_df['Kategori'] == 'Sakit Mata']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Mata']['Obat'].unique()}")

In [ ]:
# Menampilkan daftar obat untuk penyakit bagian pernapasan
print(f"Total data obat untuk penyakit bagian pernafasan murni : {drugs_df[drugs_df['Kategori'] == 'Sakit Pernafasan']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Pernafasan']['Obat'].unique()}")

In [ ]:
# Menampilkan daftar obat untuk penyakit bagian jantung
print(f"Total data obat untuk penyakit bagian jantung murni : {drugs_df[drugs_df['Kategori'] == 'Sakit Jantung']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Jantung']['Obat'].unique()}")

Total data obat untuk penyakit bagian jantung murni : 12
['Cardio Aspirin' 'Farnormin' 'Clopidogrel Bisulfate' 'Triatec' 'Concor'
 'Spironolactone' 'Vfend Voriconazole' 'Rifampicin' 'Nitrokaf Retard'
 'Amlodipine Besylate' 'Simarc 2' 'Lasix']


In [ ]:
# Menampilkan daftar obat untuk penyakit bagian pencernaan
print(f"Total data obat untuk penyakit bagian perut murni : {drugs_df[drugs_df['Kategori'] == 'Sakit Perut']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Perut']['Obat'].unique()}")

Total data obat untuk penyakit bagian perut murni : 22
['Entrostop' 'Diapet' 'Imodium' 'Polysilane Suspensi' 'Promag Suspensi'
 'Omeprazole' 'Ranitidine' 'Dulcolax' 'Laxatab' 'Lactulax' 'Ambeven'
 'Ardium 500' 'Nutrafor Wazzir' 'Lanpracid Kapsul' 'Grafazol Kaplet'
 'Zithromax' 'Floxigra' 'Ciprofloxacin Tablet' 'Flagyl Forte'
 'Metronidazole' '-' 'Duspatalin']


In [ ]:
# Menampilkan daftar obat untuk penyakit bagian pencernaan
print(f"Total data obat untuk penyakit bagian perut murni : {drugs_df[drugs_df[''] == 'Sakit Perut']['Obat'].nunique()}")
print(f"{drugs_df[drugs_df['Kategori'] == 'Sakit Perut']['Obat'].unique()}")

Total data obat untuk penyakit bagian perut murni : 22
['Entrostop' 'Diapet' 'Imodium' 'Polysilane Suspensi' 'Promag Suspensi'
 'Omeprazole' 'Ranitidine' 'Dulcolax' 'Laxatab' 'Lactulax' 'Ambeven'
 'Ardium 500' 'Nutrafor Wazzir' 'Lanpracid Kapsul' 'Grafazol Kaplet'
 'Zithromax' 'Floxigra' 'Ciprofloxacin Tablet' 'Flagyl Forte'
 'Metronidazole' '-' 'Duspatalin']


## DATA PREPARATION (Content-Based Filtering)

In [ ]:
# Membuat Salinan Dari Model Data Frame
drugs_df_recommender = drugs_df.copy()

In [ ]:
# Mengambil beberapa data yang telah melewati proses cleaning data
drugs_df_recommender = drugs_df_recommender[['Disease (Penyakit)', 'Obat', 'Deskripsi', 'Kandungan', 'Dosis', 'Aturan Pakai', 'Efek Samping', 'Toko Online 1', 'Toko Online 2', 'Kategori']]

In [ ]:
# Menampilkan informasi singkat berdasarkan kolom
drugs_df_recommender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Disease (Penyakit)  127 non-null    object
 1   Obat                127 non-null    object
 2   Deskripsi           127 non-null    object
 3   Kandungan           127 non-null    object
 4   Dosis               127 non-null    object
 5   Aturan Pakai        127 non-null    object
 6   Efek Samping        127 non-null    object
 7   Toko Online 1       127 non-null    object
 8   Toko Online 2       127 non-null    object
 9   Kategori            127 non-null    object
dtypes: object(10)
memory usage: 10.1+ KB


In [ ]:
# Menampilkan 5 data awal dan 5 data akhir
drugs_df_recommender

,Disease (Penyakit),Obat,Deskripsi,Kandungan,Dosis,Aturan Pakai,Efek Samping,Toko Online 1,Toko Online 2,Kategori
0,Migrain,Bodrex Migra,Bodrex migra merupakan obat kaplet yang mengan...,"Paracetamol 350 mg, Propyphenazone 150 mg, dan...","Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\r\nhttps://www.halodoc.com/obat-dan-v...,\n\nAlodokter:\nhttps://www.alodokter.com/alos...,Sakit Kepala
1,Migrain,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,Sakit Kepala
2,Migrain,Farsifen Plus,Farsifen plus merupakan salah satu jenis varia...,"Paracetamol, Ibuprofen, dan Kafein","Dewasa dan anak-anak berusia > 12: 1 kaplet, 3...",Dapat dikonsumsi setalah makan untuk menghinda...,Obat tersebut memiliki beberapa efek samping y...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24Klik:\nhttps://www.k24klik.com/p/farsifen-p...,Sakit Kepala
3,Tension Headache,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,Sakit Kepala
4,Tension Headache,Paracetamol,Paracetamol atau dapat dikenal sebagai asetami...,Paracetamol,"1.000 mg, 4 kali/hari, selama 7 hari",Dapat dionsumsi sebelum atau setelah makan,Obat ini jarang menimbulkan efek samping kepad...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/paracetamo...,Sakit Kepala
...,...,...,...,...,...,...,...,...,...,...
122,Disentri,Metronidazole,Obat ini bekerja dengan membunuh parasit penye...,Metronidazole,"Dosis: 3 kali sehari, selama 5–10 hari.",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/metronidaz...,Sakit Perut
123,Hepatitis,-,-,-,-,-,-,-,-,Sakit Perut
124,Irritable Bowel Syndrome (IBS),Duspatalin,Duspatalin merupakan obat yang digunakan untuk...,Mebeverine Hydrochloride 135 mg,Dosis: 1 tablet dikonsumsi sebanyak 3x sehari,Dikonsumsi 20 menit sebelum makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/duspatalin...,Sakit Perut
125,Irritable Bowel Syndrome (IBS),Dulcolax,Dulcolax adalah obat pencahar untuk mengatasi ...,Bisacodyl,Dewasa dan anak usia > 10 tahun: 1–2 tablet pe...,Dapat dikonsumsi sebelum tidur,Dulcolax memiliki beberapa efek samping yang d...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/dulcolax-t...,Sakit Perut


In [ ]:
# Membuat kolom baru yang menggabungkan 3 data menjadi 1 yang akan digunakan dalam proses cosine similarity dan menampilkan data tersebut
drugs_df_recommender['Atribut Obat'] = drugs_df_recommender['Disease (Penyakit)'] + drugs_df_recommender['Kategori'] + ", " + drugs_df_recommender['Obat'] + ", " + drugs_df_recommender['Deskripsi'] + ", "  + drugs_df_recommender['Kandungan'] + ", " + drugs_df_recommender['Dosis'] + ", " + drugs_df_recommender['Aturan Pakai']
drugs_df_recommender["Atribut Toko Online"] = drugs_df_recommender['Toko Online 1'] + ", " + drugs_df_recommender['Toko Online 2']
drugs_df_recommender

,Disease (Penyakit),Obat,Deskripsi,Kandungan,Dosis,Aturan Pakai,Efek Samping,Toko Online 1,Toko Online 2,Kategori,Atribut Obat,Atribut Toko Online
0,Migrain,Bodrex Migra,Bodrex migra merupakan obat kaplet yang mengan...,"Paracetamol 350 mg, Propyphenazone 150 mg, dan...","Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\r\nhttps://www.halodoc.com/obat-dan-v...,\n\nAlodokter:\nhttps://www.alodokter.com/alos...,Sakit Kepala,"MigrainSakit Kepala, Bodrex Migra, Bodrex migr...",Halodoc:\r\nhttps://www.halodoc.com/obat-dan-v...
1,Migrain,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,Sakit Kepala,"MigrainSakit Kepala, Panadol Extra, Panadol ex...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
2,Migrain,Farsifen Plus,Farsifen plus merupakan salah satu jenis varia...,"Paracetamol, Ibuprofen, dan Kafein","Dewasa dan anak-anak berusia > 12: 1 kaplet, 3...",Dapat dikonsumsi setalah makan untuk menghinda...,Obat tersebut memiliki beberapa efek samping y...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24Klik:\nhttps://www.k24klik.com/p/farsifen-p...,Sakit Kepala,"MigrainSakit Kepala, Farsifen Plus, Farsifen p...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
3,Tension Headache,Panadol Extra,Panadol extra merupakan varian lain dari panad...,Paracetamol 500 mg dan Kafein 65 mg,Dewasa dan Anak-anak > 12 tahun: 1 kaplet dimi...,Dapat dikonsumsi sebelum atau setelah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,Alodokter:\nhttps://www.alodokter.com/aloshop/...,Sakit Kepala,"Tension HeadacheSakit Kepala, Panadol Extra, P...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
4,Tension Headache,Paracetamol,Paracetamol atau dapat dikenal sebagai asetami...,Paracetamol,"1.000 mg, 4 kali/hari, selama 7 hari",Dapat dionsumsi sebelum atau setelah makan,Obat ini jarang menimbulkan efek samping kepad...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/paracetamo...,Sakit Kepala,"Tension HeadacheSakit Kepala, Paracetamol, Par...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
...,...,...,...,...,...,...,...,...,...,...,...,...
122,Disentri,Metronidazole,Obat ini bekerja dengan membunuh parasit penye...,Metronidazole,"Dosis: 3 kali sehari, selama 5–10 hari.",Dapat dikonsumsi sesudah makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/metronidaz...,Sakit Perut,"DisentriSakit Perut, Metronidazole, Obat ini b...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
123,Hepatitis,-,-,-,-,-,-,-,-,Sakit Perut,"HepatitisSakit Perut, -, -, -, -, -","-, -"
124,Irritable Bowel Syndrome (IBS),Duspatalin,Duspatalin merupakan obat yang digunakan untuk...,Mebeverine Hydrochloride 135 mg,Dosis: 1 tablet dikonsumsi sebanyak 3x sehari,Dikonsumsi 20 menit sebelum makan,Konsumsi obat yang berlebihan dapat menimbulka...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/duspatalin...,Sakit Perut,"Irritable Bowel Syndrome (IBS)Sakit Perut, Dus...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
125,Irritable Bowel Syndrome (IBS),Dulcolax,Dulcolax adalah obat pencahar untuk mengatasi ...,Bisacodyl,Dewasa dan anak usia > 10 tahun: 1–2 tablet pe...,Dapat dikonsumsi sebelum tidur,Dulcolax memiliki beberapa efek samping yang d...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...,K24klik:\nhttps://www.k24klik.com/p/dulcolax-t...,Sakit Perut,"Irritable Bowel Syndrome (IBS)Sakit Perut, Dul...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...


In [ ]:
unique_obat = drugs_df["Obat"].unique().tolist()
kandungan_arr = []
atribut_obat_arr = []
dosis_arr = []
aturan_pakai_arr = []
atribut_toko_online_arr = []

for obat in unique_obat :
    # Ambil baris-baris yang sesuai dengan obat saat ini dari drugs_df
    kandungan_terkait = drugs_df[drugs_df["Obat"] == obat]["Kandungan"].unique()
    kandungan_arr.append(kandungan_terkait)
    kandungan_terkait_str = ", ".join(kandungan_terkait) if len(kandungan_terkait) > 0 else ""

    atribut_terkait = "; ".join((drugs_df[drugs_df['Obat'] == obat]['Disease (Penyakit)'].values)) + ", " + str(obat) + ", " + kandungan_terkait_str
    atribut_obat_arr.append(atribut_terkait)

    # Ambil dosis pertama yang terkait dengan obat tersebut
    dosis_arr.append(drugs_df["Dosis"].iloc[0] if not drugs_df.empty else None)

    # Ambil aturan pakai pertama yang terkait dengan obat tersebut
    aturan_pakai_arr.append(drugs_df["Aturan Pakai"].iloc[0] if not drugs_df.empty else None)

    # Ambil atribut toko online pertama yang terkait dengan obat tersebut dari drugs_df_recommender
    # Pastikan untuk mengambil dari drugs_df_recommender setelah kolom Atribut Toko Online dibuat
    toko_online_terkait = " dan ".join(drugs_df["Toko Online 1"].values) + " dan " + " dan ".join(drugs_df["Toko Online 2"].values)
    atribut_toko_online_arr.append(drugs_df_recommender[drugs_df_recommender['Obat'] == obat]['Atribut Toko Online'].iloc[0] if not drugs_df_recommender[drugs_df_recommender['Obat'] == obat].empty else None)


drugs_df_recommender_2 = pd.DataFrame({
    "Obat": unique_obat,
    "Kandungan": kandungan_arr,
    "Dosis": dosis_arr,
    "Aturan Pakai": aturan_pakai_arr,
    "Atribut Obat": atribut_obat_arr,
    "Atribut Toko Online": atribut_toko_online_arr
})

print (kandungan_arr[0])

['Paracetamol 350 mg, Propyphenazone 150 mg, dan Kafein 50 mg']


In [ ]:
# Menampilkan 5 data teratas
drugs_df_recommender_2.head()

,Obat,Kandungan,Dosis,Aturan Pakai,Atribut Obat,Atribut Toko Online
0,Bodrex Migra,"[Paracetamol 350 mg, Propyphenazone 150 mg, da...","Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,"Migrain, Bodrex Migra, Paracetamol 350 mg, Pro...",Halodoc:\r\nhttps://www.halodoc.com/obat-dan-v...
1,Panadol Extra,[Paracetamol 500 mg dan Kafein 65 mg],"Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,"Migrain; Tension Headache, Panadol Extra, Para...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
2,Farsifen Plus,"[Paracetamol, Ibuprofen, dan Kafein]","Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,"Migrain, Farsifen Plus, Paracetamol, Ibuprofen...",Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
3,Paracetamol,[Paracetamol],"Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,Tension Headache; Ensefalitis; Otitis Media; G...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...
4,Ibuprofen,[Ibuprofen],"Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,Tension Headache; Gendang telinga pecah; Larin...,Halodoc:\nhttps://www.halodoc.com/obat-dan-vit...


In [ ]:
# Menampilkan informasi ringkas terkait keseluruhan dataset
drugs_df_recommender_2.describe()

,Obat,Kandungan,Dosis,Aturan Pakai,Atribut Obat,Atribut Toko Online
count,88,88,88,88,88,88
unique,88,77,1,1,88,86
top,Bodrex Migra,[-],"Dewasa: 1 kaplet, 3x sehari",Dapat dikonsumsi sesudah makan,"Migrain, Bodrex Migra, Paracetamol 350 mg, Pro...","-, -"
freq,1,4,88,88,1,3


In [ ]:
# print(f"Total data obat yang terduplikasi : {drugs_df_recommender_2['Obat'].duplicated().sum()}")
# print(f"Total data kandungan yang terduplikasi : {drugs_df_recommender_2['Kandungan'].duplicated().sum()}")
# print(f"Total data yang terduplikasi pada atribut obat : {drugs_df_recommender_2['Atribut Obat'].duplicated().sum()}")

In [ ]:
# Menampilkan seluruh penyakit yang dapat diatasi dengan obat tersebut
", ".join((drugs_df[drugs_df['Obat'] == 'Paracetamol']['Disease (Penyakit)'].values))

'Tension Headache, Ensefalitis, Otitis Media, Gendang telinga pecah, Barotrauma'

## Modeling

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(drugs_df_recommender_2['Atribut Obat']) # Menyesuaikan kosa kata

TfidfVectorizer()

In [ ]:
tfidf.get_feature_names_out().shape

(330,)

In [ ]:
print(tfidf.get_feature_names_out())

['000' '10' '100' '135' '150' '160' '197' '20' '200' '204' '240' '25'
 '250' '275' '30' '335' '350' '400' '50' '500' '6000' '62' '65' '72' '75'
 '80' '84' '85' '90' 'acyclovir' 'akilen' 'akut' 'alat' 'alopecia'
 'aluminium' 'ambeien' 'ambeven' 'amlodipine' 'amoxicillin' 'amoxsan'
 'angina' 'angustifolia' 'ardium' 'asiatica' 'asma' 'aspirin' 'atenolol'
 'atorvastatin' 'attapulgite' 'avelox' 'azithromycin' 'baclofen'
 'bamgetol' 'bantu' 'barotrauma' 'batuk' 'besylate' 'betahistine'
 'betaserc' 'biji' 'bisacodyl' 'bisoprolol' 'bisulfate' 'blumei' 'bodrex'
 'bowel' 'breathy' 'bromida' 'bronkitis' 'buah' 'budesonide' 'calcium'
 'carbamazepine' 'cardio' 'cefixime' 'cendo' 'centella' 'chalazion'
 'chloramphenicol' 'cholestor' 'ciprofloxacin' 'citrate' 'citrus'
 'clopidogrel' 'cochlear' 'cold' 'coleus' 'concor' 'cordifoliae' 'cream'
 'curcuma' 'curcumae' 'dabigatran' 'dalam' 'dan' 'daun' 'delima' 'dengan'
 'dexamethasone' 'dextromethorphan' 'diapet' 'diare' 'dihydrate' 'diosmin'
 'disentri' 'd

In [ ]:
tfidf_matrix = tfidf.transform(drugs_df_recommender_2['Atribut Obat'])

In [ ]:
tfidf_matrix.shape

(88, 330)

In [ ]:
cosim_matrix = cosine_similarity(tfidf_matrix)

In [ ]:
cosim_matrix.shape

(88, 88)

In [ ]:
cosine_df = pd.DataFrame(cosim_matrix,
                         columns=drugs_df_recommender_2['Obat'].values.tolist(),
                         index=drugs_df_recommender_2['Obat'].values.tolist())

In [ ]:
cosine_df.loc["Paracetamol"]

,Paracetamol
Bodrex Migra,0.125916
Panadol Extra,0.315618
Farsifen Plus,0.166431
Paracetamol,1.000000
Ibuprofen,0.448567
...,...
Floxigra,0.000000
Flagyl Forte,0.000000
Metronidazole,0.000000
-,0.000000


In [ ]:
def rekomendasi_obat(penyakit, df, cosine_df):
    # Mengambil baris pertama untuk penyakit yang relevan
    obat_utama_row = df[df['Disease (Penyakit)'] == penyakit].iloc[[0]]

    # Dapatkan nama obat yang sebenarnya untuk pencarian di cosine_df
    obat_utama_name = obat_utama_row['Obat'].iloc[0]

    print(f"Penyakit yang diderita\t: {penyakit}")
    print("-" * 100)

    # Menyalin seluruh data jika DataFrame penyakit sama dengan penyakit yang sedang diamati
    kandidat_rekomendasi_obat = df[(df['Disease (Penyakit)'] == penyakit)].copy()

    # Menampilkan pesan ketika penyakit yang diderita adalah Hepatitis
    if penyakit == "Hepatitis":
      print(f"Harap Konsultasikan ke dokter terlebih dahulu")
      return

    # Menghitung nilai cosine similarity dari obat utama
    kandidat_rekomendasi_obat['Similarity'] = kandidat_rekomendasi_obat.apply(
        lambda row: cosine_df.loc[obat_utama_name, row['Obat']], axis=1
    )

    # Urutkan data rekomendasi obat berdasarkan nilai Similarity
    rekomendasi = kandidat_rekomendasi_obat.sort_values(by='Similarity', ascending=False)

    # Menampilkan rekomendasi obat
    print("Rekomendasi Obat Serupa:")
    for index, rekomendasi_item in enumerate(rekomendasi.itertuples(), 1):
        print(f"\n{index}.\t{rekomendasi_item.Obat}")
        print(f"Deskripsi Obat\t: \n{rekomendasi_item.Deskripsi}\n")  # Memanggil Data Pada Kolom Deskripsi
        print(f"Kandungan\t: {rekomendasi_item.Kandungan}\n") # Memanggil Data Pada Kolom Kandungan
        print(f"Dosis\t\t: \n{rekomendasi_item.Dosis}\n")  # Memanggil Data Pada Kolom Dosis
        print(f"Aturan Pakai\t: \n{rekomendasi_item._6}\n") # Memanggil Data Pada Kolom Ke-6 (Aturan Pakai)
        print(f"Efek Samping\t: \n{rekomendasi_item._7}\n") # Memanggil Data Pada Kolom Ke-7 (Efek Samping)
        print(f"Toko Online 1\t: \n{rekomendasi_item._8}\n")  # Memanggil Data Pada Kolom Ke-8 (Toko Online 1)
        print(f"Toko Online 2\t: \n{rekomendasi_item._9}\n")  # Memanggil Data Pada Kolom Ke-9 (Toko Online 2)
        print("-" * 100)

In [ ]:
with pd.option_context('display.max_rows', 5,
                       'display.max_columns', 5,
                       'display.precision', 3,
                       ):
    print(cosine_df)

               Bodrex Migra  Panadol Extra  ...    -  Duspatalin
Bodrex Migra          1.000          0.391  ...  0.0       0.059
Panadol Extra         0.391          1.000  ...  0.0       0.042
...                     ...            ...  ...  ...         ...
-                     0.000          0.000  ...  1.0       0.000
Duspatalin            0.059          0.042  ...  0.0       1.000

[88 rows x 88 columns]


In [ ]:
rekomendasi_obat("Chalazion", drugs_df_recommender, cosine_df)

Penyakit yang diderita	: Chalazion
----------------------------------------------------------------------------------------------------
Rekomendasi Obat Serupa:

1.	Erlamycetin Salep Mata
Deskripsi Obat	: 
Erlamycetin salep mata merupakan varian lain dari obat erlamycetin yang dapat digunakan sebagai antibiotik untuk menangani infeksi bakteri yang terjadi pada bagian mata (konjungtivitis bakterial) dan keluhan lainnya, seperti mata merah, gatal, belekan, hingga sensitif terhadap cahaya. Kandungan chloramphenicol pada obat ini akan mengganggu pembentukan protein yang dibutuhkan oleh bakteri untuk berkembang biak dan bertahan hidup. Obat ini merupakan obat yang ampuh digunakan untuk kondisi mata yang mengalami infeksi lebih berat dari biasanya dan obat ini hanya bisa didapatkan setelah pasien melakukan konsultasi dengan dokter

Kandungan	: Chloramphenicol 10 mg

Dosis		: 
Dewasa dan anak-anak berusia ≥ 2 tahun: Oleskan erlamysetin salep mata pada yang mengalami infeksi sebanyak 3 - 4x se

In [ ]:
cosine_df.to_json("list_obat.json")

In [ ]:
drugs_df_recommender.to_json("list_kandungan.json")

## Evaluasi